# Field ON vs Field OFF — Food Odor (Easy Environment)

**The question:** Does the pheromone field improve foraging with partial observability?

**Previous results:**
- Full food obs (K=5): Field OFF wins (field redundant)
- No food obs: both die (can't bootstrap)
- Food odor on 20×20: both nearly die, but Field ON shows 1.3 deliveries vs 0.0

**This run:** Easier environment so agents survive long enough to LEARN:
- `grid_size=10` (4× more food encounters)
- `num_food=20` (2× denser)
- `starting_energy=400` (2× exploration time)
- Food odor (exp(-dist/4)) — vague smell, no exact positions
- 2 conditions × 3 seeds × 3M steps

**Run all cells and check the verdict at the bottom.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

REPO_DIR = '/content/emergence-lab'
GITHUB_USERNAME = 'imashishkh21'

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/{GITHUB_USERNAME}/emergence-lab.git {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull origin main

os.chdir(REPO_DIR)
!pip install -e ".[dev]" -q

import jax
print(f"JAX devices: {jax.devices()}")

In [ ]:
from src.configs import Config

TOTAL_STEPS = 3_000_000
NUM_SEEDS = 3
CHECKPOINT_BASE = '/content/drive/MyDrive/emergence-lab/food_odor_easy_env'


def build_config(field_on: bool) -> Config:
    """Build config for experiment. Easy environment for learning bootstrap."""
    config = Config()

    # Easy environment — agents must survive long enough to learn
    config.env.grid_size = 10       # small grid: 4x more encounters
    config.env.num_agents = 8
    config.env.num_food = 20        # dense food: 2x more sources

    # Partial observability: smell food, don't see it
    config.env.food_obs_enabled = False
    config.env.food_odor_enabled = True
    config.env.food_odor_lambda = 4.0

    # Evolution — generous energy for learning phase
    config.evolution.enabled = True
    config.evolution.max_agents = 32
    config.evolution.starting_energy = 400   # 2x more exploration time
    config.evolution.max_energy = 400        # match starting energy
    config.evolution.food_energy = 100
    config.evolution.reproduce_threshold = 120
    config.evolution.reproduce_cost = 80
    config.evolution.mutation_std = 0.01

    # Training
    config.train.total_steps = TOTAL_STEPS
    config.train.num_envs = 32
    config.train.num_steps = 128
    config.log.wandb = False
    config.log.save_interval = 0

    # No extras
    config.nest.patch_scaling_enabled = False
    config.field.adaptive_gate = False

    if field_on:
        # Ch0 recruitment ON, Ch1 OFF
        config.field.channel_diffusion_rates = (0.5, 1.0, 0.0, 0.0)
        config.field.channel_decay_rates = (0.05, 1.0, 0.0, 0.0)
    else:
        # ALL channels instant decay = no field
        config.field.channel_decay_rates = (1.0, 1.0, 1.0, 1.0)
        config.field.channel_diffusion_rates = (0.0, 0.0, 0.0, 0.0)

    return config


CONDITIONS = [
    ("field_OFF", build_config(field_on=False)),
    ("field_ON", build_config(field_on=True)),
]

print(f"Conditions: {[c[0] for c in CONDITIONS]}")
print(f"Seeds per condition: {NUM_SEEDS}")
print(f"Steps per seed: {TOTAL_STEPS:,}")
print(f"Checkpoint base: {CHECKPOINT_BASE}")
print(f"grid_size: {CONDITIONS[0][1].env.grid_size}")
print(f"num_food: {CONDITIONS[0][1].env.num_food}")
print(f"starting_energy: {CONDITIONS[0][1].evolution.starting_energy}")
print(f"food_odor_enabled: {CONDITIONS[0][1].env.food_odor_enabled}")
print()
print("Easy environment: small grid, dense food, generous energy.")
print("Agents smell food but can't see it. Field ON should amplify foraging.")

In [ ]:
import gc
import time
import numpy as np
from src.training.parallel_train import ParallelTrainer

all_results = {}

for condition_name, config in CONDITIONS:
    print(f"\n{'='*60}")
    print(f"RUNNING: {condition_name}")
    print(f"{'='*60}")

    checkpoint_dir = f"{CHECKPOINT_BASE}/{condition_name}"
    seed_ids = list(range(NUM_SEEDS))

    steps_per_iter = config.train.num_envs * config.train.num_steps * config.evolution.max_agents
    num_iterations = max(1, TOTAL_STEPS // steps_per_iter)

    print(f"Steps/iter: {steps_per_iter:,}")
    print(f"Iterations: {num_iterations}")

    try:
        t0 = time.time()
        trainer = ParallelTrainer(
            config=config,
            num_seeds=NUM_SEEDS,
            seed_ids=seed_ids,
            checkpoint_dir=checkpoint_dir,
            master_seed=42,
        )

        metrics = trainer.train(
            num_iterations=num_iterations,
            checkpoint_interval_minutes=30,
            resume=False,
            print_interval=5,
        )

        elapsed = time.time() - t0

        all_results[condition_name] = {
            'metrics': metrics,
            'time': elapsed,
            'success': True,
        }

        print(f"\n{condition_name} completed in {elapsed/60:.1f} minutes")

    except Exception as e:
        print(f"FAILED: {e}")
        import traceback
        traceback.print_exc()
        all_results[condition_name] = {'success': False, 'error': str(e)}

    finally:
        try:
            del trainer
        except Exception:
            pass
        gc.collect()
        try:
            if hasattr(jax, 'clear_caches'):
                jax.clear_caches()
        except Exception:
            pass

print("\n" + "="*60)
print("ALL CONDITIONS COMPLETE")
print("="*60)

In [ ]:
import numpy as np
from scipy import stats

print("="*60)
print("RESULTS COMPARISON: Field ON vs Field OFF")
print("="*60)

# Extract per-seed metrics
results = {}
for name, result in all_results.items():
    if not result.get('success'):
        print(f"\n{name}: FAILED — {result.get('error', 'unknown')}")
        continue

    m = result['metrics']
    rewards = np.array(m.get('mean_reward', [0.0]), dtype=float)
    population = np.array(m.get('population_size', [0.0]), dtype=float)
    pickups = np.array(m.get('num_pickups', [0.0]), dtype=float)
    deliveries = np.array(m.get('num_deliveries', [0.0]), dtype=float)
    delivery_rate = np.where(pickups > 0, deliveries / pickups, 0.0)
    per_agent = np.where(population > 0, deliveries / population, 0.0)

    results[name] = {
        'reward': rewards,
        'population': population,
        'pickups': pickups,
        'deliveries': deliveries,
        'delivery_rate': delivery_rate,
        'per_agent': per_agent,
        'time': result.get('time', 0),
    }

    print(f"\n--- {name} ({len(rewards)} seeds) ---")
    print(f"  Reward:        {np.mean(rewards):.4f} +/- {np.std(rewards):.4f}")
    print(f"  Population:    {np.mean(population):.2f} +/- {np.std(population):.2f}")
    print(f"  Pickups:       {np.mean(pickups):.1f} +/- {np.std(pickups):.1f}")
    print(f"  Deliveries:    {np.mean(deliveries):.1f} +/- {np.std(deliveries):.1f}")
    print(f"  Delivery rate: {np.mean(delivery_rate):.3f} +/- {np.std(delivery_rate):.3f}")
    print(f"  Per-agent del: {np.mean(per_agent):.3f} +/- {np.std(per_agent):.3f}")
    print(f"  Time:          {result.get('time', 0)/60:.1f} min")

# Statistical comparison
if 'field_ON' in results and 'field_OFF' in results:
    print("\n" + "="*60)
    print("STATISTICAL COMPARISON")
    print("="*60)

    on = results['field_ON']
    off = results['field_OFF']

    for metric_name in ['deliveries', 'delivery_rate', 'per_agent', 'population']:
        on_vals = on[metric_name]
        off_vals = off[metric_name]

        if len(on_vals) >= 2 and len(off_vals) >= 2:
            t_stat, p_val = stats.ttest_ind(on_vals, off_vals, equal_var=False)
            diff = np.mean(on_vals) - np.mean(off_vals)
            pooled_std = np.sqrt((np.std(on_vals)**2 + np.std(off_vals)**2) / 2)
            cohens_d = diff / pooled_std if pooled_std > 0 else 0.0
            winner = 'ON' if diff > 0 else 'OFF'
            sig = '***' if p_val < 0.01 else '**' if p_val < 0.05 else '*' if p_val < 0.1 else 'ns'
        else:
            t_stat, p_val, cohens_d, winner, sig = 0, 1, 0, '?', 'ns'

        print(f"\n  {metric_name}:")
        print(f"    ON:  {np.mean(on_vals):.3f} +/- {np.std(on_vals):.3f}")
        print(f"    OFF: {np.mean(off_vals):.3f} +/- {np.std(off_vals):.3f}")
        print(f"    diff: {diff:+.3f} | p={p_val:.4f} {sig} | d={cohens_d:.2f} | winner={winner}")

    # Final verdict
    on_del = np.mean(on['deliveries'])
    off_del = np.mean(off['deliveries'])
    _, p_del = stats.ttest_ind(on['deliveries'], off['deliveries'], equal_var=False)

    print("\n" + "="*60)
    if on_del > off_del and p_del < 0.05:
        pct = (on_del - off_del) / off_del * 100 if off_del > 0 else float('inf')
        print(f"VERDICT: FIELD ON WINS (+{pct:.1f}%, p={p_del:.4f})")
        print("Pheromone stigmergy improves foraging in carry/deliver task.")
    elif off_del > on_del and p_del < 0.05:
        pct = (off_del - on_del) / on_del * 100 if on_del > 0 else float('inf')
        print(f"VERDICT: FIELD OFF WINS (+{pct:.1f}%, p={p_del:.4f})")
        print("Pheromone field does not help (or hurts) in this config.")
    else:
        print(f"VERDICT: NO SIGNIFICANT DIFFERENCE (p={p_del:.4f})")
        print("Cannot distinguish field ON from OFF at this sample size.")
    print("="*60)